In [1]:
%%capture
!pip install youtube_dl

In [58]:
import requests
import json
import time

api= "45929254f44e4117aa634667a2c345b8"
upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'

headers_auth_only = {"authorization" : api}

headers = {"authorization" : api,
           "content-type" : "application/json" }

chunk_size = 5242880 # 5mb

def upload(filename):
    def read_file(filename):
        with open(filename, "rb") as f:
            while True:
                data = f.read(chunk_size)
                if not data:
                    break
                yield data

    upload_response = requests.post(upload_endpoint,headers = headers_auth_only, data = read_file(filename))
    return upload_response.json["upload_url"]

def transcribe(audio_url,sentiment_analysis):
    transcript_request = {
        "audio_url" : audio_url,
        "sentiment_analysis" :sentiment_analysis
    }
    transcript_response = requests.post(transcript_endpoint,headers = headers , json = transcript_request)
    return transcript_response.json()["id"]

def poll(transcprit_id):
    polling_endpoint = transcript_endpoint + "/" + transcprit_id
    polling_response = requests.get(polling_endpoint,headers = headers)
    return polling_response.json()

def get_transcription_result_url(url,sentiment_analysis):
    transcribe_id = transcribe(url,sentiment_analysis)
    while True:
        data = poll(transcribe_id)
        if data["status"] == "completed":
            return data,None
        elif data["status"] == "error":
            return data, data["error"]
    print("Waiting For 5 Seconds...")
    time.sleep(5)

def save_transcript(url,title,sentiment_analysis = False):
    data, error =get_transcription_result_url(url,sentiment_analysis) 

    if data:
        filename = title + ".txt"
        with open(filename,"w") as f:
            f.write(data["text"])
        
        if sentiment_analysis:
            filename = title + "_sentiments.json"
            with open(filename,"w") as f:
                sentiments = data["sentiment_analysis_results"]
                json.dump(sentiments,f,indent = 4)

        print("Transcript Saved ...")


    elif error:
        print("Error : " ,error)

In [59]:
import youtube_dl

ydl = youtube_dl.YoutubeDL()
url = "https://youtu.be/e-kSGNzu0hM"

def get_video_infos(url):
    with ydl:
        result = ydl.extract_info(url,
                                download = False)
        
    if "entries" in result:
        return result["entries"][0]
    return result

def get_audio_url(video_info):
    for f in video_info["formats"]:
        #print(f)
        #print(f["ext"])
        #print(f["url"])
        #print("".center(50,"*"))

        # Just take sound no video !!!
        if f["ext"] == "m4a":
            print(f["url"])
            return f["url"]

if __name__ =="__main__":
    video_info = get_video_infos(url)
    audio_url = get_audio_url(video_info)
    print(audio_url)

[youtube] e-kSGNzu0hM: Downloading webpage
https://rr2---sn-q4fl6nzy.googlevideo.com/videoplayback?expire=1655984293&ei=RfyzYoHmG5eakwaIraFg&ip=34.125.196.161&id=o-ANquz_hoiIboMp5SLEN30LTo0RzwX19NFniJyDTaxQ-Z&itag=140&source=youtube&requiressl=yes&mh=j-&mm=31%2C26&mn=sn-q4fl6nzy%2Csn-5go7ynld&ms=au%2Conr&mv=u&mvi=2&pl=21&spc=4ocVC_HnSU5oM_Vxf8IHUt0f-TT_iaA&vprv=1&mime=audio%2Fmp4&ns=WazgrLEKehSjg1nGf_uR5zQG&gir=yes&clen=6429687&dur=397.246&lmt=1655864023403212&mt=1655962358&fvip=1&keepalive=yes&fexp=24001373%2C24007246&c=WEB&txp=4532434&n=vRQTynxNB_jeUJpwrk&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Cmime%2Cns%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRAIfNSTo8LhViiRSh9trQOIpKtN392caawuqZp_M9knNRwIhAI8mBC3shrqwP8yad6zxrwBZ6-R-o9e1lzWgCW9qvkuF&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl&lsig=AG3C_xAwRAIgGxkBHdQALGEKln5qMwTWNtQRh8DZHnAp2aZoP8emE0wCIBgThIrQRzaCFtQOZHeRUFe2wcrRJbj3FIVLQn1fsotw
https://rr2---sn-q4fl6nzy.googlevideo.com/videoplayback?expire=1655984293

In [62]:
def save_video_sentiments(url,sentiment_analysis = True):
    video_infos = get_video_infos(url)
    audio_url = get_audio_url(video_infos)
    title = video_infos["title"]
    title = title.strip().replace(" ", "_")
    #title = "data/" + title
    print(title)
    save_transcript(audio_url,title,sentiment_analysis = True)


if __name__ == "__main__":
    save_video_sentiments(url)

[youtube] e-kSGNzu0hM: Downloading webpage
https://rr1---sn-q4flrnss.googlevideo.com/videoplayback?expire=1655984874&ei=if6zYvzbO4avkwbFp6_IDA&ip=34.125.196.161&id=o-AL50M6cqjOi4ujKudcoCspaZSjsWl7Ea_--Yc9LcRQfG&itag=140&source=youtube&requiressl=yes&mh=j-&mm=31%2C26&mn=sn-q4flrnss%2Csn-5goeen7r&ms=au%2Conr&mv=u&mvi=1&pl=21&spc=4ocVC_zkMRspx6NSD2U_1rzvWs8cJKk&vprv=1&mime=audio%2Fmp4&ns=RDUWiaiJf_7vqdaEpUr067oG&gir=yes&clen=6429687&dur=397.246&lmt=1655864023403212&mt=1655962358&fvip=1&keepalive=yes&fexp=24001373%2C24007246&c=WEB&txp=4532434&n=QqfWDyP8SoFnEw75po&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Cmime%2Cns%2Cgir%2Cclen%2Cdur%2Clmt&sig=AOq0QJ8wRAIgY5EQOF-xr7IgpImeA8LVzBJCNdub37LozMGrifosQ9ACICr4eWYu45tEpHtpYShD7rYWTxJdRn1BivJffDynfhA6&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl&lsig=AG3C_xAwRAIgXatcXOnwkjeDuWt8O2dZ5OKTE4D9GsJYwkUIv_9vktcCIHCOBRJNjSJiyz-yUUZRivFk-dWrydJk63FOb2omaILd
iPhone_13_Review:_Pros_and_Cons
Transcript Saved ...


In [67]:
if __name__ == "__main__":

    file_path = "/content/iPhone_13_Review:_Pros_and_Cons_sentiments.json"
    with open(file_path,"r") as f:
        data = json.load(f)

    positives = []
    negatives = []
    neutrals = []

    for result in data:
        text = result["text"]
        if result["sentiment"] == "POSITIVE":
            positives.append(text)
        elif result["sentiment"] == "NEGATIVE":
            negatives.append(text)
        else:
            neutrals.append(text)

    n_pos = len(positives)
    n_neg = len(negatives)
    n_neu = len(neutrals)
    
    p_r = n_pos / (n_pos + n_neg)
    n_r = n_neg / (n_pos + n_neg)   

    print(f"Positive Ratio : {p_r:.3f}")
    print(f"Negative Ratio : {n_r:.3f}")
    

Positive Ratio : 0.867
Negative Ratio : 0.133
